In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.models import Sequential
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

training_data = pd.read_csv("data/fashion-mnist_train.csv")
labels = training_data['label']
training_data.drop(["label"],axis=1,inplace=True)
test_data = pd.read_csv("data/fashion-mnist_test.csv")
img_array = np.array(training_data.iloc[3].values, dtype=np.uint8)
img_reshape = img_array.reshape((28,28))
image = Image.fromarray(img_reshape)

labels.info

2024-07-07 16:08:03.000453: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-07 16:08:03.032266: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-07 16:08:03.139967: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-07 16:08:03.284135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 16:08:03.415202: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

<bound method Series.info of 0        2
1        9
2        6
3        0
4        3
        ..
59995    9
59996    1
59997    8
59998    8
59999    7
Name: label, Length: 60000, dtype: int64>

## Data preparation

In [26]:
# Label encoding
encoder = OneHotEncoder(sparse_output=False)
encoded_labels = encoder.fit_transform(labels.to_frame())
print(encoded_labels)
# Data normalization
training_data = (training_data / 255)
print(training_data.shape)
training_data.isna()

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


## Train

In [ ]:
conv_num_filters = 8
conv_filter_size = 3
pool_size = 2

model = Sequential(
    Conv2D(num_filters=conv_num_filters,filter_size=conv_filter_size,input_shape = (28,28,1)),
    MaxPooling2D(pool_size=pool_size),
    Flatten(),
    Dense(10, activation='softmax') # 10 neurons = 10 classes
)